In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib notebook



In [5]:
#loading data

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [7]:
# saving the PasserengerId for final

passengerId=test.PassengerId

In [8]:
# merge train and test

titanic=train.append(test,ignore_index=True)

# creating index

# doubt1

train_idx=len(train)
test_idx=len(titanic)-len(test)

In [16]:
print(train_idx)
print(test_idx)

891
891


In [17]:
titanic.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [18]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [20]:
#To check if any column has null
print(titanic.isnull().sum())
#To get those columns:
print(titanic[titanic.isnull().T.any().T])


Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64
       Age Cabin Embarked      Fare  \
0     22.0   NaN        S    7.2500   
2     26.0   NaN        S    7.9250   
4     35.0   NaN        S    8.0500   
5      NaN   NaN        Q    8.4583   
7      2.0   NaN        S   21.0750   
8     27.0   NaN        S   11.1333   
9     14.0   NaN        C   30.0708   
12    20.0   NaN        S    8.0500   
13    39.0   NaN        S   31.2750   
14    14.0   NaN        S    7.8542   
15    55.0   NaN        S   16.0000   
16     2.0   NaN        Q   29.1250   
17     NaN   NaN        S   13.0000   
18    31.0   NaN        S   18.0000   
19     NaN   NaN        C    7.2250   
20    35.0   NaN        S   26.0000   
22    15.0   NaN        Q    8.0292   
24     8.0   NaN        S   21.0750   
25    

In [23]:
titanic.drop('PassengerId',axis=1,inplace=True)

In [24]:
# to verify that PassengerId is gone, sorry old habit :)
titanic.keys()

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'Pclass', 'Sex',
       'SibSp', 'Survived', 'Ticket'],
      dtype='object')

In [25]:
#creating title for feature engineering

titanic['Title']=titanic.Name.apply(lambda name:name.split(',')[1].split('.')[0].strip())

titanic.head()

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket,Title
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171,Mr
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599,Mrs
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0,STON/O2. 3101282,Miss
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,1.0,113803,Mrs
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,0.0,373450,Mr


In [27]:
print(titanic.Title.unique())

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer' 'Dona']


In [28]:
# normalize the titles
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}